# Memory Management

Memory management is a core concept in programming and refers to how programs allocate and deallocate memory during execution. In Python, memory management ensures that objects are properly stored, tracked, and removed when no longer needed. This allows Python programs to run efficiently without consuming unnecessary memory.

Python has an **automatic memory management system** that relies on techniques like **reference counting**, **garbage collection**, and **memory pooling** to handle memory allocation and deallocation.

---

## Key Concepts in Python Memory Management

### 1. Memory Allocation

When you create a new object in Python, memory is allocated to store the object. Python uses a built-in memory manager to allocate memory for all objects.

---

### 2. Reference Counting

Python uses **reference counting** as its primary memory management mechanism. Every object has an associated reference count that tracks how many references (or pointers) point to that object.

#### How Reference Counting Works:

1. When an object is created, its reference count is set to **1**.
2. When a new variable points to the same object, its reference count is **incremented**.
3. When a reference goes out of scope, the reference count is **decremented**.
4. If the reference count reaches **0**, the object is no longer used and can be deallocated.

---

### 3. Garbage Collection

While reference counting handles most cases of memory management, it cannot handle **circular references** (e.g., objects referencing each other). For these cases, Python uses a **garbage collector** to free up memory.


In [1]:
class A:
    def __init__(self):
        self.b = None


class B:
    def __init__(self):
        self.a = None


a = A()
b = B()
a.b = b
b.a = a

#### Circular References

- In this example, `a` and `b` reference each other, creating a circular reference.
- The garbage collector detects this situation and clears the objects from memory.

#### 4. Memory Pools

- Python optimizes memory usage by using memory pools.
- For small, frequently-used data types (like integers and short strings), Python reuses memory to reduce overhead.

### 1. Reference Counting

#### How Reference Counting Works:

- Python maintains a count of references to every object in memory.
- Objects are deallocated when their reference count reaches zero.


In [2]:
import sys

a = [1, 2, 3]
## 2 (one reference from 'a' and one from getrefcount())
print(
    sys.getrefcount(a)
)  # Shows the number of references to 'a' (includes temporary references)

b = a
print(
    sys.getrefcount(a)
)  # Reference count increases because now `b` also points to 'a'

del b
print(sys.getrefcount(a))  # Reference count decreases when `b` goes out of scope

2
3
2


### Garbage Collection

Python includes a cyclic garbage collector to handle reference cycles. Reference cycles occur when objects reference each other, preventing their reference counts from reaching zero.

Python uses the gc module to clean up memory by identifying and deallocating objects no longer used, especially in the case of circular references.


In [3]:
import gc


class Node:
    def __init__(self, name):
        self.ref = None
        self.name = name
        print(f"Object {self.name} created")

    def __del__(self):
        print(f"Object {self.name} deleted")


node1 = Node("obj1")
node2 = Node("obj2")
node1.ref = node2
node2.ref = node1

del node1
del node2

Object obj1 created
Object obj2 created


In [4]:
print(gc.collect())  # Force garbage collection

Object obj1 deleted
Object obj2 deleted
4


### The gc Module

Python provides a gc module for interacting with the garbage collector. You can manually trigger garbage collection, disable it, or inspect objects tracked by the garbage collector.


In [5]:
import gc

# Disable garbage collection
gc.disable()

# Enable garbage collection
gc.enable()

# Force garbage collection
gc.collect()

### Get garbage collection stats
print(gc.get_stats())

### get unreachable objects
print(gc.garbage)

[{'collections': 166, 'collected': 1370, 'uncollectable': 0}, {'collections': 15, 'collected': 367, 'uncollectable': 0}, {'collections': 3, 'collected': 4, 'uncollectable': 0}]
[]


## Memory Profiling

To optimize memory usage, it's important to monitor memory consumption during program execution. Python provides tools like:

- **`sys.getsizeof()`**: Returns the size of an object in bytes.
- **`psutil`**: A cross-platform library to monitor system resources.
- **`memory_profiler`**: A Python package for tracking memory usage line-by-line.


In [6]:
import sys

my_list = [1, 2, 3, 4, 5]
print(f"Size of my_list: {sys.getsizeof(my_list)} bytes")

Size of my_list: 104 bytes


## Memory Management Best Practices

To ensure optimal memory usage, consider the following best practices:

1. **Avoid Circular References**  
   Circular references lead to memory not being deallocated until the garbage collector runs. Use weak references (via the `weakref` module) when circular references are necessary.


In [7]:
import weakref


class Node:
    def __init__(self):
        self.ref = None


node1 = Node()
node2 = Node()
node1.ref = weakref.ref(node2)
node2.ref = weakref.ref(node1)

2. **Limit the Use of Global Variables**  
   Global variables can lead to unintended retention of memory. Use function-local variables or context managers where possible.

3. **Use Generators Instead of Lists**  
   When dealing with large data sets, use generators. They produce data lazily, saving memory because data is computed on demand.


In [8]:
# List (consumes all memory at once)
my_list = [x for x in range(1000000)]

# Generator (saves memory by computing values lazily)
my_gen = (x for x in range(1000000))

4. **Explicitly Delete Unused Objects**  
   Use the `del` keyword to explicitly delete variables no longer in use to free memory.


In [9]:
data = [1, 2, 3]
del data

5. **Profile and Optimize Memory Usage**  
   Use tools like `gc`, `memory_profiler`, or `tracemalloc` to monitor memory consumption and find memory leaks.

---

## Python's Built-in Modules for Memory Management

1. **gc (Garbage Collection)**

   - Monitors and manages circular references.
   - Allows you to control garbage collection behavior.

2. **sys.getsizeof()**

   - Returns the memory size of an object in bytes.

3. **tracemalloc**
   - Tracks memory allocations and helps debug memory usage issues.


In [10]:
import tracemalloc

tracemalloc.start()

a = [1] * 1000000
print(tracemalloc.get_traced_memory())

del a
print(tracemalloc.get_traced_memory())

(8001041, 8019157)
(2817, 8020835)


In [11]:
def create_list():
    return [i for i in range(10000)]


def main():
    tracemalloc.start()

    create_list()

    snapshot = tracemalloc.take_snapshot()
    top_stats = snapshot.statistics("lineno")

    print("[ Top 10 ]")
    for stat in top_stats[::]:
        print(stat)


main()

[ Top 10 ]
d:\Workspace\python-concepts-master\.venv\Lib\site-packages\IPython\core\compilerop.py:86: size=11.6 KiB, count=108, average=110 B
C:\Users\ribhav.jain\AppData\Local\Programs\Python\Python312\Lib\tokenize.py:537: size=5600 B, count=100, average=56 B
C:\Users\ribhav.jain\AppData\Local\Programs\Python\Python312\Lib\codeop.py:126: size=2135 B, count=24, average=89 B
C:\Users\ribhav.jain\AppData\Local\Programs\Python\Python312\Lib\json\decoder.py:353: size=1701 B, count=21, average=81 B
d:\Workspace\python-concepts-master\.venv\Lib\site-packages\zmq\sugar\socket.py:805: size=1056 B, count=6, average=176 B
d:\Workspace\python-concepts-master\.venv\Lib\site-packages\jupyter_client\session.py:1057: size=1040 B, count=5, average=208 B
d:\Workspace\python-concepts-master\.venv\Lib\site-packages\IPython\core\compilerop.py:174: size=924 B, count=12, average=77 B
d:\Workspace\python-concepts-master\.venv\Lib\site-packages\zmq\sugar\attrsettr.py:45: size=846 B, count=18, average=47 B
d:\

## Summary

### Memory Management Concepts in Python

| **Concept**            | **Description**                                           |
| ---------------------- | --------------------------------------------------------- |
| **Reference Counting** | Tracks the number of references pointing to an object.    |
| **Garbage Collection** | Cleans up circular references.                            |
| **Memory Pools**       | Optimizes memory reuse for frequently-used small objects. |
| **Weak References**    | Avoid circular references by using weak references.       |
| **tracemalloc**        | Tracks memory allocations and identifies memory leaks.    |
| **sys.getsizeof()**    | Returns the memory consumption of an object.              |
